In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
drugall = pd.read_csv('../../big datasets/drug_data_to_use.csv')

In [ ]:
drug_neg = drugall.iloc[1017:,:]
classtox = []

for index, row in drug_neg.iterrows():
    classtox.append(0)
    
drug_neg.insert(loc=1, column='Class', value=classtox)



drug_pos = drugall.iloc[:1017,:]
classdrug = []

for index, row in drug_pos.iterrows():
    classdrug.append(1)
    
drug_pos.insert(loc=1, column='Class', value=classdrug)

drug_pos.head()

In [ ]:
drugall = drug_pos.append(drug_neg)

In [ ]:
drugall.to_csv('../../big datasets/drugall_with_classes.csv')

In [ ]:
drug_pos_features = drug_pos.iloc[:,2:]
drug_pos_features.head()
drug_pos_class = drug_pos['Class']

drug_neg_features = drug_neg.iloc[:,2:]
drug_neg_class = drug_neg['Class']

In [ ]:
x_train_neg, x_test_neg, y_train_neg, y_test_neg = train_test_split(drug_neg_features, drug_neg_class, test_size=0.2, random_state=12)
x_train, x_test, y_train, y_test = train_test_split(drug_pos_features, drug_pos_class, test_size=0.2, random_state=12)


In [ ]:
x_train = x_train.append(x_train_neg)
x_test = x_test.append(x_test_neg)
y_train = y_train.append(y_train_neg)
y_test = y_test.append(y_test_neg)
all_train = x_train.join(y_train)
all_test = x_test.join(y_test)
len(y_test) == len(x_test)
len(x_train) == len(y_train)

In [ ]:
all_train.drop(all_train.select_dtypes(['object']), inplace=True, axis=1)
all_train.astype('float64')

all_test.drop(all_test.select_dtypes(['object']), inplace=True, axis=1)
all_test.astype('float64')

all_train.head(), all_train.info()

In [ ]:
with pd.option_context('mode.use_inf_as_null', True):
    all_train = all_train.dropna(how='all', axis=1)
    all_test = all_test.dropna(how='all',axis=1)

In [ ]:
all_train = all_train[np.isfinite(all_train).all(1)]
all_test = all_test[np.isfinite(all_test).all(1)]
all_train.shape[1] == all_test.shape[1]

In [ ]:
y_train = all_train['Class']
x_train = all_train.drop(['Class'], axis=1)

x_test = all_test.drop(['Class'], axis=1)
y_test = all_test['Class']

In [ ]:
from sklearn.preprocessing import StandardScaler
xscaler = StandardScaler().fit(x_train)
x_train = xscaler.transform(x_train)
testscaler = StandardScaler().fit(x_test)
x_test = testscaler.transform(x_test)

In [ ]:
sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

In [ ]:
print(len(x_train),len(y_train),len(x_train_res),len(y_train_res))